<img align="left" src = "https://www.linea.org.br/brand/linea-logo-color.svg" width=130  style="padding: 40px">  
<img align="left" src = "https://cdn2.webdamdb.com/1280_c3PXjCZbPM23.png" width=220> 

<font size=5> **Photo-z Server** Tutorial Notebook
 </font>

Authors: Julia Gschwend, Luigi Silva, Cristiano Singulani <br> 
Contact: [Julia Gschwend](mailto:julia@linea.org.br) <br> 
Last verified run: **2025-Nov-10**

# Introduction

The Photo-z (PZ) Server is an online service for the LSST Community to create, host, and share lightweight PZ-related data products. It is developed and maintained by LIneA as part of the BRA-LIN in-kind contribution to the Rubin Observatory. The service is hosted at the Brazilian IDAC, with access restricted to the LSST Community via [Rubin Science Platform (RSP)](https://data.lsst.cloud/) login credentials. For more information about the PZ Server and other contributions related to photometric redshifts, please visit the [BRA-LIN's description page](https://linea-it.github.io/pz-lsst-inkind-doc/).

The PZ Server has two main user interfaces: the website and the API, which is accessed through the `pzserver` Python library.

This notebook provides instructions for new users on how to use the `pzserver` Python library, including examples for all available functions and methods. Website usage documentation is available on [LIneA's Documentation for Users webpage](https://docs.linea.org.br/en/sci-platforms/pz_server.html).

# Getting Started

## Installation

The PZ Server's Python library is available on **pip** as `pzserver`.

```
$ pip install --upgrade --no-cache-dir pzserver
```

Note 1: Depending on your Jupyter Notebook or Lab version, you might need to restart the kernel to use the newly installed library.

Note 2: If you are installing it in the RSP Notebook Aspect using the LSST kernel, you may see warnings about dependency versions. These warnings should not affect library functionality. If you encounter any issues, please contact the [PZ Server team](mailto:pzserver-admin@linea.org.br).

Since `pzserver` is still in development, make sure to install the latest stable version and bypass any cached versions.

In [ ]:
pip install --upgrade --no-cache-dir pzserver

## Import library

In [ ]:
from pzserver import PzServer 

## The PzServer class 

The `PzServer` class connects to the PZ Server database and provides access to data and metadata. To create a `PzServer` object, users must be authorized using an API token, which can be generated from the menu at the top right corner of the [PZ Server website](https://pzserver.linea.org.br/).

<img src="https://docs.linea.org.br/images/pz-server-token-menu.png" width=150pt align="top"/> <img src="https://docs.linea.org.br/images/pz-server-generate-token.png" width=350pt />

Paste your API token by replacing the placeholder below:

In [ ]:
pz_server = PzServer(token="<your token here>")

API tokens can be reused indefinitely. However, generating a new token will automatically invalidate any previously issued one.

For convenience, you can save your API token in a text file, such as **token.txt**. This file is already included in the `.gitignore` of this repository to prevent accidental exposure. 

In [ ]:
# with open('token.txt', 'r') as file:
#   token = file.read()
# pz_server = PzServer(token=token)

<font color="red"><strong>API tokens MUST NOT be shared. Users are responsible for keeping their tokens private.</strong></font>

--- 
# Basic methods

## Query general info

The `pz_server` object created above provides access to data and metadata stored in the PZ Server. It also includes additional methods that help users navigate the available content. 

Methods with the prefix `get_` return the results of queries to the PZ Server database as Python dictionaries and are best suited for programmatic use (see the [API documentation page](https://linea-it.github.io/pzserver/html/index.html) for details). In contrast, methods with the prefix `display_` show results as styled [Pandas DataFrames](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), optimized for use in Jupyter Notebooks. *(Note: column names may differ slightly in the display version.)*

For example, you can display the list of supported product types with a short description: 

In [ ]:
pz_server.display_product_types()

(for a full explanation of product types and their upload requirements, refer to the [product types section](https://docs.linea.org.br/en/sci-platforms/pz_server.html#data-product-types) in the PZ Server documentation)

Display the list of data releases currently available:

In [ ]:
pz_server.display_releases()

Display all available data products: 

<font color='green'><strong>WARNING:</strong> This list may grow quickly as the survey progresses. It is recommended to enable scrolling in the notebook cell output.</font>


In [ ]:
pz_server.display_products_list() 

The information about product type, users, and releases shown above can be used to filter the data products relevant to your search. To do this, the method `display_products_list` takes a dictionary as an argument that maps product attributes to their corresponding values.

In [ ]:
pz_server.display_products_list(filters={"release": "DP0.2", 
                                         "product_type": "Training Set"})

It also works if you type a string pattern that is part of the value.

In [ ]:
pz_server.display_products_list(filters={"product_type": "estimates"})

To fetch the results of a search and assign them to a variable, just change the prefix `display_` to `get_`:

In [ ]:
search_results = pz_server.get_products_list(filters={"product_type": "training results"}) 
search_results

## Display metadata

<font size=4>**product_id** and **internal_name**</font>

All data products stored on the PZ Server are identified by their unique **product_id** number or **internal_name**, which is created automatically at the moment of upload by concatenating the **product_id** with the name given by its owner (replacing blank spaces with "_", converting to lowercase, and removing special characters) (e.g., `30_simple_training_set`). 

<font size=4>Display the metadata of a data product</font>

The metadata of a given data product is all the information available about it, including what the user provided on the upload form.

The `PzServer` method `get_product_metadata()` returns a dictionary with the attributes stored in the PZ Server for a given data product, identified by its **id** or **internal_name**. For use in a Jupyter notebook, the equivalent method `display_product_metadata()` shows the results in a formatted table.


In [ ]:
product_id = 30
pz_server.display_product_metadata(product_id)

## Download

To download any data product stored in the PZ Server, use the method `download_product`, providing the **product_id** or **internal_name** and the path where it will be saved (the default is the current folder). This method downloads a compressed .zip file containing all files uploaded by the user, including data, auxiliary files, and description files. Let's try it with a small data product.

In [ ]:
pz_server.download_product(product_id, save_in=".")

## Get (Load to Memory)

Instead of downloading the files, the `pzserver` library also allows users to retrieve the contents of a given data product to work directly in memory using the method `get_product()`. This feature is available only for tabular data, such as redshift catalogs and training sets.

By default, the method `get_product` returns an `astropy.Table`, which can be easily converted into a `pandas.DataFrame`. Let's see an example with the data product mentioned above:

In [ ]:
data = pz_server.get_product(30).to_pandas()
data

In [ ]:
data.describe()

## Share

All data products uploaded to the PZ Server are immediately available and visible to all PZ Server users, i.e., people with RSP credentials, through the PZ Server website or Python library. One way to share a data product is by providing the product's URL, which leads to the product's download page. The URL is formed by concatenating the PZ Server website address, **/products/**, and the **internal_name**:

For example, for the data downloaded above:

In [ ]:
internal_name = pz_server.get_product_metadata(product_id)['internal_name']
url = f'https://pzserver.linea.org.br/product/{internal_name}'
url

---
# Advanced methods

## Upload 


The default method to upload a data product to the PZ Server is using the upload form on the website. Alternatively, the `pzserver` Python library can be used to send data products to the host service.

The first step is to prepare a dictionary with the relevant information about your data product. For example:

In [ ]:
data_to_upload = {
    "name":"example upload via lib",
    "product_type": "redshift_catalog",  # Product type 
    "release": None, # LSST release, use None if not LSST data 
    "main_file": "./examples/upload_example.csv", # full path 
    "auxiliary_files": ["./examples/upload_example.html", "./examples/upload_example.ipynb"] # full path
    #"auxiliary_files": [] # you must give a empty list if you don't have any auxiliary_files
}

Then, execute the `upload` method from the `pz_server` class, providing the product details dictionary as a `kwargs` argument.

In [ ]:
upload = pz_server.upload(**data_to_upload)  
upload.product_id

The upload is not complete yet! The step below only starts the process.  
For **Reference Redshift Catalogs** and **Training Sets**, users must provide the mapping of their column names.

```python
columns = {
    "<your-RA-column-name>": "RA",
    "<your-Dec-column-name>": "Dec",
    "<your-z-column-name>": "z"
}
```

For instance: 

In [ ]:
columns = {
    "RA": "RA",
    "DEC": "Dec",
    "Z": "z"
}

upload.make_columns_association(columns)

At this point, you can proceed to save it.

In [ ]:
upload.save()

Review the results of your upload:

In [ ]:
pz_server.display_product_metadata(upload.product_id) 

## Update

To modify an existing product, you first need to define the product object.

In [ ]:
#product_object = pz_server.get_product_object(<product_id>)
product_object = pz_server.get_product_object(upload.product_id)

The attributes of this product can be viewed.

In [ ]:
product_object.attributes

<font size=4>Adding an auxiliary file</font>

You can add an auxiliary file and/or a description file by providing their paths. The difference between these two files is that the description file is an HTML file displayed on the product details page of the PZ Server website (e.g., an exported notebook).  

In [ ]:
# product_object.attach_auxiliary_file("<path_to_auxiliary_file>")
# product_object.attach_description_file("<path_to_description_file>")

Now, you can check if the uploads were completed correctly.

In [ ]:
# product_object.get_auxiliary_files()

In [ ]:
# product_object.get_description_files()

<font size=4>Update the description</font>

You can also update the product description as shown in `pzserver`.

In [ ]:
product_object.update_description("test update description")

In [ ]:
product_object.attributes['description']

In [ ]:
pz_server.display_product_metadata(upload.product_id) 

## Delete

To delete a data product along with all its files (main and auxiliary), use the method `delete_product`.

<font color="red">**BE CAREFUL! THIS ACTION CANNOT BE UNDONE!**</font>

In [ ]:
#pz_server.delete_product(<product_id>)
pz_server.delete_product(upload.product_id)

# PZ Server Pipelines 

In addition to PZ-related data hosting and curation services, PZ Server also provides tools to help users prepare training data for PZ algorithms. The pipeline *Training Set Maker* uses the data partitioning method [HATS](https://hats.readthedocs.io/en/stable/) and the Python framework [LSDB](https://docs.lsdb.io/en/stable/) (both developed by [LINCC](https://lsstdiscoveryalliance.org/programs/lincc/)) as the cross-matching back-end engine, coupled with a user interface on the PZ Server website connected to IDAC-Brazil's high-performance computing infrastructure. 

With *Training Set Maker*, users can create training sets by matching objects from a reference redshift catalog available on the server with objects from an LSST Object catalog. The reference catalog can be previously uploaded by the user or created as a combination of multiple redshift catalogs by another pipeline, called *Combine Redshift Catalogs*.


<img src="https://docs.linea.org.br/images/pz-server-tsm.png" width="600" style="display: block; margin: auto;" />

Both pipelines are executed as asynchronous processes triggered either from the PZ Server website or directly from Python scripts using the `pzserver` library, with the outputs automatically registered as new data products.

### Combine Redshift Catalogs

The pipeline **Combine Redshift Catalogs (CRC)** combines multiple redshift catalogs into a single table and registers it as a new data product on the PZ Server. It was designed to help **aggregate multiple samples from individual surveys** into a single catalog before they are associated with LSST data through spatial cross-matching.

It **optionally deduplicates** catalog entries based on the ```z_flag_homogenized``` and ```instrument_type_homogenized``` columns, which can be automatically generated for some specific supported surveys (the ```survey_name``` column is required in this case) or manually embedded by the user in their catalogs. See the **documentation at https://docs.linea.org.br/en/sci-platforms/pz_server_crc.html** and the **currently supported automatic translations at https://github.com/linea-it/pzserver_pipelines/blob/main/combine_redshift_dedup/flags_translation.yaml**. 

> **Note:** all the **SITCOMTN-154 reference redshift catalogs are supported**, considering that they have a flag column which is quality like, that is, with float values between 0 and 1.

On the PZ Server website, go to **PZ Server Pipelines > Combine Redshift Catalogs**, fill in the submission form with relevant metadata, such as the name for the new reference redshift catalog to be created and a short description, select the catalogs to include by marking at least two checkboxes, and press the **Run** button.

![CRC](images/crc.png)

Let's now see how to run the pipeline through the lib.

Start creating a ```crc``` process object instance by providing a name (string) for the new reference redshift catalog to the method.

In [ ]:
#crc = pz_server.combine_redshift_catalogs(<new product's name>)
crc = pz_server.combine_redshift_catalogs("crc_example")

In [ ]:
type(crc)

Check status of the ```crc``` process.

In [ ]:
crc.check_status()

In [ ]:
crc.summary()

View the current pipeline parameters and modify them if desired.

> **Note:** For now, the ```flags_translation.yaml``` file needs to be in a location that the ```app.orch``` application can access, in the LIneA storage system. We don't recommend changing it for now. 
> **Future work:** Users will be able to upload customized versions ```flags_translation.yaml``` to change default's translation dictionary. More detailed instructions on how to do this will be available soon.
 

In [ ]:
crc.pipeline.parameters

Update configuration parameters

The deduplication options available are (`combine_type`): 
- `concatenate`
- `concatenate_and_mark_duplicates`
- `concatenate_and_remove_duplicates`
  


In [ ]:
cfg = {
    "combine_type": "concatenate_and_mark_duplicates",
    "flags_translation_file": "/data/apps/app.orch/dev/pipelines/combine_redshift_dedup/flags_translation.yaml",
}

crc.set_config(cfg)
crc.summary()

Then, add at least two individual reference redshift ratalogs to be included in the sample using the ```append_catalog``` method. These catalogs must already exist in the PZ Server, and their internal names identify them. Let's browse the reference redshift catalogs available and choose from the list:

In [ ]:
pz_server.display_products_list(filters={"product_type": "Reference Redshift Catalog"})

Let us add, as an example, the 2dFGRS, 6dFGS and NED spectroscopic catalogs.

In [ ]:
crc.append_catalog(60) # 2dFGRS 
crc.append_catalog(64) # 6dFGS 
crc.append_catalog(70) # NED 

Let's check the summary of ```crc``` attributes, now updated with the input catalogs added above:

In [ ]:
crc.summary()

Now, use the method run to submitt the process as an asychronous job to the PZ Server's back-end.

In [ ]:
crc.run()

Still during the process, we can check the ```id``` and the ```internal_name``` of the output data product with the summary method.

In [ ]:
crc.summary()

Or get it from the object ```crc```:

In [ ]:
catalog_id = crc.output.get('id') 
catalog_id

In [ ]:
catalog_name = crc.output.get('internal_name') 
catalog_name

Let's check if the process is done, if the status is 'Successful', we can move on to the next cell.

In [ ]:
crc.check_status()

The possible values for the process status are *Pending*, *Queued*, *Running*, *Successful*, and *Failed*.  
If the status is *Failed*, please contact the Photo-z Server administrators at **helpdesk@linea.org.br**.  

Once the status changes to *Successful*, the new redshift catalog named ```crc_example``` becomes available for download or can be directly retrieved into memory.

In [ ]:
my_new_specz_catalog = pz_server.get_product(catalog_name)

In [ ]:
type(my_new_specz_catalog)

We can convert to a pandas dataframe and inspect.

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 10)

my_new_specz_catalog_df = my_new_specz_catalog.to_pandas()

In [ ]:
my_new_specz_catalog_df.head()

In [ ]:
my_new_specz_catalog_df.describe()

In [ ]:
import matplotlib.pyplot as plt

my_new_specz_catalog_df.plot(x="ra", y="dec", kind="scatter")  
plt.xlabel("R.A. (degrees)")
plt.ylabel("Dec. (degrees)")
plt.tight_layout()

### Training Set Maker 

<font color=red> Pipeline under development. </font>

The release will be announced on the Community Forum soon!


---
# User Feedback

Did you find a bug? Is something important missing?

Send your feedback to us [via email](mailto:helpdesk@linea.org.br), or feel free to [open an issue](https://github.com/linea-it/pzserver/issues/new) in the PZ Server library repository on GitHub.
